## Packages & Jeu de données

In [65]:
import pandas as pd

# Chargement du jeu de données 
full_data = pd.read_csv('Data/TRAIN_FULL/data_usagers.csv', low_memory=False)

full_data.head()

,place,catu,grav,sexe,trajet,locp,actp,etatp,an_nais,senc,...,surf,infra,situ,env1,secu1,secu2,secu3,motor,vma,id_usager
0,1,1,0,0,5,9,9,-1,1939,0.0,...,1.0,0.0,1.0,0.0,1,8,8,NaN,NaN,NaN
1,10,3,0,1,5,3,3,2,2008,0.0,...,1.0,0.0,1.0,0.0,8,8,8,NaN,NaN,NaN
2,1,1,0,1,5,9,9,-1,1994,0.0,...,1.0,0.0,1.0,99.0,8,8,8,NaN,NaN,NaN
3,2,2,0,1,9,9,9,-1,1992,0.0,...,1.0,0.0,1.0,99.0,1,8,8,NaN,NaN,NaN
4,1,1,0,1,5,9,9,-1,1976,0.0,...,1.0,0.0,1.0,99.0,2,8,8,NaN,NaN,NaN


In [66]:
full_data.shape

(1275851, 54)

In [67]:
def convert_float_to_int(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    
def convert_float_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype(int)
    data_frame[col_name] = data_frame[col_name].astype('category')
    
def convert_str_to_cat(data_frame, col_name):
    data_frame[col_name] = data_frame[col_name].astype('category')

### Caractéristiques

Les fichiers caractéristiques contiennent globalement des informations relatives aux contextes de l'accident d'un point de vue conditions météorologiques, temporelles, etc.

#### jour, mois, an, hrmm 

- **jour** : jour de l'accident.
- **mois** : mois de l'accident.
- **an** : année de l'accident.
- **hrmn** : heure et minutes de l'accident.

Ces informations sont cruciales, car des périodes pourraient être plus propices à des accidents graves. Un exemple serait qu'en hiver, une visibilité plus faible est observée ou bien un taux de suicide plus important pourrait expliquer une hausse de la gravité des accidents. En somme, sans fournir de justifications, ces informations peuvent être pertinentes.

In [68]:
# Traitement sur les années pour harmoniser la forme des étiquettes [2012 et non 12 par exemple]
full_data.loc[(full_data['an'] >= 12) & (full_data['an'] <= 18), 'an'] += 2000

# Harmonise hr:mn de type string en int
def convert_to_int(time_str):
    if isinstance(time_str, str):
        if ':' in time_str:
            hours, minutes = map(int, time_str.split(':'))
            return hours * 100 + minutes
        else:
            return int(time_str)
    else:
        return time_str


full_data['hrmn'] = full_data['hrmn'].apply(convert_to_int)

# Converti en valeur numérique
convert_float_to_int(full_data, 'hrmn')

# Traitement des minutes 
def generate_intervals(start, end, step):
    intervals = []
    for i in range(start, end, step):
        intervals.append((i, i + step))
    return intervals

# Generate intervals automatically for each 100 units
intervals = generate_intervals(0, 2400, 100)
simple_values = list(range(len(intervals)))

# Function to map values to simple values based on intervals
def map_to_simple_value(value):
    for i, interval in enumerate(intervals):
        if interval[0] <= value < interval[1]:
            return simple_values[i]
    return None  # Return None if value is outside all intervals

# Apply the mapping function to the column and create a new column with the simple values
full_data['hrmn'] = full_data['hrmn'].apply(map_to_simple_value)

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'jour')
convert_float_to_cat(full_data, 'mois')
convert_float_to_cat(full_data, 'an')
convert_float_to_cat(full_data, 'hrmn')

#### lum

Apporte une information importante sur les conditions d'éclairage lors de l'accident. L'absence d'éclairage peu entraîner une perte de visibilité et amener à des répercussions importantes tandis qu'une meilleure visibilité permet d'éviter des drames. Les conditions sont classées de **"*1*"** à **"*5*"** sans relation d'ordre.

On observe un très faible nombre de valeurs non comprises au sein de l'intervalle régulier [**1**, **2**, **3**, **4**, **5**]. Ainsi, il est décidé de supprimer ces lignes.

In [69]:
# Suppression des valeurs -1
full_data = full_data[full_data['lum'] != -1]
full_data = full_data.dropna(subset=['lum'])

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'lum')

#### dep, com, agg

- **dep** : numéro du département répertorié à l'aide du code INSEE suivi d'un 0.
- **com** : numéro de commune donné par un code INSEE comportant 3 chiffres calés à droite.
- **agg** : valeur binaire indiquant si l'accident s'est produit au sein ou en dehors d'une agglomération.

Il est vrai que certaines communes peuvent être porteuses davantage que d'autres d'accidents de la route. Cependant, afin d'éviter une construction de modèle trop complexe, il est décidé d'éliminer les communes pour ne conserver que les valeurs des départements. En effet, une tendance départementale apportera suffisamment d'informations au vu des nombreuses variables disponibles.

Pour ce qui est de la valeur d'agglomération, il y a ici un indice fort intéressant, car il peut notamment traduire la vitesse des usagers. Il est donc décidé de le conserver.

Les valeurs des départements sont relativement mal respectées. Ainsi, il est nécessaire d'apporter un traitement tout particulier à ce dernier. Si un département est de la forme **"*01*"**, **"*001*"**, ou encore **"*040*"**, il faut supprimer le **"*0*"** à gauche. Dans un second temps, si un département à **"*3*"** chiffres apparaît et fini par un **"*0*"**, alors ce département ne peut pas exister. On supposera que la suppression du **'*0*'** à droite donne le vrai numéro du département. Enfin, les départements **"*201*"** et **"*202*"** seront respectivement remplacés par **"*2A*"** et **"*2B*"**.


In [70]:
# Suppression de la colonne com 
full_data.drop(labels=['com'], axis=1, inplace=True)

# Binarisation de la valeur d'agglomération
full_data['agg'] = full_data['agg'].replace({2: 0})

# Harmonisation des départements
# Supprime les 0 superflus à gauche
full_data['dep'] = full_data['dep'].apply(lambda x: x.lstrip('0'))

# Cas des départements inexistants (100, 970...)
full_data['dep'] = full_data['dep'].apply(lambda x: x[:-1] if len(x) == 3 and x.endswith('0') else x)

# Remplace les valeurs pour la Corse
full_data['dep'] = full_data['dep'].replace({'201': '2A', '202': '2B'})

# En cas d'autres valeurs de département, supprime les lignes
full_data = full_data[full_data['dep'].isin(map(str, range(1, 96))) | full_data['dep'].isin(['971', '972', '974', '976'])]

# Converti en variable catégorielle
convert_str_to_cat(full_data, 'dep')
convert_float_to_cat(full_data, 'agg')

#### int

Fourni des informations relatives aux différents types d'intersections. Il est intéressant de se pencher sur l'incidence des intersections en matière d'accident et si un rôle est joué, si la nature des incidents est importante ou non dans la gravité de ces derniers. Les valeurs [**-1**, **0**, **9**] apportent la même information : l'intersection n'est pas connue. Il sera donc décidé d'harmoniser les possibilités sous cette forme : [**1**, **2**, **3**, **4**, **5**, **6**, **7**, **8**, **9**].

In [73]:
# Harmonise les données
full_data['int'] = full_data['int'].replace({-1: 9, 0: 9})

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'int')

#### atm

Cette variable fournit des informations relatives aux conditions atmosphériques lors de l'accident. Outre un travail de normalisation entre les différentes années, il est évident que les conditions climatiques impactent les performances de conduite (perte de visibilité...).

Les valeurs comme **"*Nan*"**, **"*-1*"**, ou encore **"*0*"** ont le même rôle que la valeur **"*9*"** (Aucune information sur la situation).

In [76]:
# Harmonise les données
full_data['atm'] = full_data['atm'].replace({-1: 9, 0: 9}).fillna(9)

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'atm')

#### col

Le type de collision est fort intéressant pour pouvoir déterminer la gravité d'un accident. En effet, un véhicule percuté côté conducteur aura plus de chance de blesser gravement le conducteur que le passager à sa droite. Ainsi, une étroite corrélation pourrait apparaître entre ces variables. Globalement, les référentiels sont similaires, nous harmoniseront simplement les valeurs traduisant une absence de renseignement sur l'accident : [**-1**, **0**, **NAN**, **8**]. On exclura **"*6*"** du fait que malgré le manque d'information sur la collision, il apporte l'information qu'une collision a eu lieu.

In [79]:
# Harmonise les données
full_data['col'] = full_data['col'].replace({-1: 8, 0: 8}).fillna(8)

# Converti en variable catégorielle
convert_float_to_cat(full_data, 'col')

#### adr, lat, long, gps

- **adr** : il est difficile de composer avec cette variable tant l'hétérogénéité est importante dans les valeurs saisies. De plus, sa pertinence n'est pas démontrée quand on dispose d'ores et déjà de la latitude et de la longitude de l'accident. Cette variable sera donc naturellement supprimée.
- **lat** : latitude de l'accident. Combinée à la longitude, cela pourrait en tant que variable numérique être utile pour créer des zones à accidents graves ou moins graves (carrefour en ville où de faibles accidents se produisent fréquemment par exemple).
- **long** : longitude de l'accident. Combinée à la latitude, cela pourrait en tant que variable numérique être utile pour créer des zones à accidents graves ou moins graves (carrefour en ville où de faibles accidents se produisent fréquemment par exemple).
- **gps** : supprimé dans les dernières versions. Cependant, les valeurs de longitudes et latitudes indiquent d'ores et déjà des zones géographiques. Cette variable sera donc naturellement supprimée.

Hélas, les données de latitude et longitude sont trop peu qualitatives pour être exploitées sans un travail important. Nous décidons de supprimer ces colonnes pour l'instant. Cependant, nous nous réservons le droit de les retravailler plus tard afin d'améliorer potentiellement le modèle.

In [85]:
# Suppression des colonnes adr, gps, lat, long 
full_data.drop(labels=['adr', 'gps', 'lat', 'long'], axis=1, inplace=True)

In [86]:
full_data.to_csv('Data/TRAIN_FULL/data_usagers_caracteristiques.csv', index=False)

Désormais, toutes les variables propres aux usagers ont été traitées, on sauvegarde une version intermédiaire de ce fichier.